In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200615']:
    print('----------------------------------------------------------------')
    print(y)
        
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    in_dex = [16, 300, 852, 905]
    
    print('----------------------------------------------------------------')
    print('SH index data:')

    readPath = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SH\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs == 16) | (dateLs == 300) | (dateLs == 852) | (dateLs == 905)]
    index = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        df = pd.read_csv(i)
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        index += [df]
    index = pd.concat(index).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    for i in range(1, 6):
        if i == 1:
            index["bid" + str(i) + "p"] = index["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            index["ask" + str(i) + "p"] = index["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            index["bid" + str(i) + "q"] = index["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            index["ask" + str(i) + "q"] = index["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            index["bid" + str(i) + "p"] = index["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            index["ask" + str(i) + "p"] = index["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            index["bid" + str(i) + "q"] = index["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            index["ask" + str(i) + "q"] = index["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    index = index.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    index["time"] = (index["QuotTime"] - int(y)*1000000000).astype(np.int64)
    
    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["StockID"].isin(in_dex)) & (index["time"] >= 91500000) & (index["time"] <= 150000000)]

    data2["close"] = data2["close"].round(4)
    data1["close"] = data1["close"].round(4)
    data2["openPrice"] = data2["openPrice"].round(4)
    data1["openPrice"] = data1["openPrice"].round(4)
    data2["cum_amount"] = data2["cum_amount"].round(1)
    data1["cum_amount"] = data1["cum_amount"].round(1)

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["source"].count()
    n2 = test["IOPV"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["IOPV"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["source"])])
    
#     del index
#     del logSH
#     del data1
#     del data2
#     del test
#     del data1_1
#     del data2_1


----------------------------------------------------------------
20200615
----------------------------------------------------------------
SH index data:
0:00:00.599397


KeyboardInterrupt: 

In [10]:
logSH[(logSH["StockID"] == 852) & (logSH["cum_volume"] < 58730807)][["StockID", "time", "cum_volume", "cum_amount", "close", "openPrice"]].tail(5)

,StockID,time,cum_volume,cum_amount,close,openPrice
4537152,300,104600000,58645833,9.727207e+10,3997.7273,3982.9672
4540191,300,104604000,58645833,9.727207e+10,3997.7273,3982.9672
4541254,300,104605000,58667229,9.730843e+10,3997.4831,3982.9672
4544834,300,104610000,58697942,9.736877e+10,3997.7139,3982.9672
4548855,300,104615000,58697942,9.736877e+10,3997.7139,3982.9672


In [18]:
index[(index["StockID"] == 300) & (index["cum_volume"] <= 58817979)][["StockID", "time", "cum_volume", "cum_amount", "close", "openPrice"]].tail(10)

,StockID,time,cum_volume,cum_amount,close,openPrice
4855,300,104544210,58538187,9.711939e+10,3996.7742,3982.9672
4856,300,104549830,58564761,9.716328e+10,3997.2846,3982.9672
4857,300,104554870,58602709,9.720785e+10,3997.1534,3982.9672
4858,300,104559200,58645833,9.727207e+10,3997.7273,3982.9672
4859,300,104604870,58667229,9.730843e+10,3997.4831,3982.9672
4860,300,104609890,58697942,9.736877e+10,3997.7139,3982.9672
4861,300,104609890,58697942,9.736877e+10,3997.7139,3982.9672
4862,300,104619850,58766367,9.747897e+10,3998.1809,3982.9672
4863,300,104624920,58785698,9.750816e+10,3998.1254,3982.9672
4864,300,104629210,58817979,9.755953e+10,3997.7440,3982.9672


In [7]:
data1["cum_amount"].astype('str').apply(lambda x: len(str(x.split('.')[1]))).unique()

array([1, 6, 5], dtype=int64)

In [5]:
data1[(data1["StockID"] == 300) & (data1["cum_volume"] == 2895389)]["cum_amount"].iloc[0]

4438748603.400001